# Demo of the Model on Differential Equations

In [1]:
import os

import numpy
import pandas
import sympy
import torch

from src.envs import build_env
from src.envs.sympy_utils import simplify_equa_diff
from src.envs.char_sp import InvalidPrefixExpression
from src.envs.sympy_utils import simplify
from src.model import build_modules
from src.utils import AttrDict
from src.utils import to_cuda

Use the Model to Find the solutions to first and second order differential equations.

## First Order Differential Equations (ODE 1)

Procedure:
1. Start from a bivariate function $F(x,c)$, that will be the equation solution, that can be solved in $c$.
2. Solve $F(x,c)$ in $c$.
3. Differentiate in $x$.
4. Simplify the final form.

### Build Environment - Reload Model

Get Trained Model:

In [2]:
model_path = '../models/differential-equations/ode1.pth'
assert os.path.isfile(model_path)

Set the Parameters for Environment and for the Model:

 Environment:
- **env_name**: SymPy character environment.
- **int_base**: integer representation base.
- **balanced**: balanced representation (base > 0).
- **positive**: do not sample negative numbers.
- **precision**: float numbers precision.
- **n_variables**: number of variables in expressions (between 1 and 4).
- **n_coefficients**: number of coefficients in expressions (between 0 and 10).
- **leaf_probs**: leaf probabilities of being a variable, a coefficient, an integer, or a constant.
- **max_len**: maximum sequences length.
- **max_int**: maximum integer value.
- **max_ops**: maximum number of operators.
- **max_ops_G**: maximum number of operators for G in IBP.
- **clean_prefix_expr**: clean prefix expressions (f x -> Y, derivative f x x -> Y').
- **rewrite_functions**: rewrite expressions with a given SymPy function.
- **tasks**: tasks to run (prim_fwd, prim_bwd, prim_ibp, ode1, ode2).
- **operators**: considered operators (add, sub, mul, div), followed by (unnormalized) sampling probabilities.


 Model:
- **cpu**: run on CPU.
- **emb_dim**: embedding layer size.
- **n_enc_layers**: number of transformer layers in the encoder.
- **n_dec_layers**: number of transformer layers in the decoder.
- **n_heads**: number of transformer heads.
- **dropout**: dropout.
- **attention_dropout**: dropout in the attention layer.
- **sinusoidal_embeddings**: use sinusoidal embeddings.
- **share_inout_emb**: share input and output embeddings.
- **reload_model**: reload a pretrained model.

In [3]:
params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

Set the Environment with SymPy:

In [4]:
env = build_env(params)
x = env.local_dict['x']
f = env.local_dict['f']
c = sympy.Symbol('c')

The primary components of the model are one encoder and one decoder network. The encoder turns each item into a
corresponding hidden vector containing the item and its context. The decoder reverses the process, turning the vector
into an output item, using the previous output as the input context.

Build Model Modules:

In [5]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare Differential Equation Input and its Solution

Declare a bivariate function $F(x,c)$, that will be the equation solution, that can be solved in c:

In [6]:
# y_infix = 'x*log(c/x)'
# y_infix = '((4+(2*x))**(-1))*((x+(sin(x)))*((x**4)+(c*(x**(-1)))))'
# y_infix = 'exp(c+((sqrt(cos(x)))+(acos(2*x))))'
y_infix = '3+((c*(x**(-1)))+(sin(tanh(cos(x)))))'

Converts **y_infix** to a type that can be used inside SymPy:

In [7]:
y = sympy.sympify(y_infix, locals=env.local_dict)
y

c/x + sin(tanh(cos(x))) + 3

Solve $y$ in $c$:

In [8]:
solve_c = sympy.solve(f(x) - y, c, check=False, simplify=False)
c = solve_c[0]
c

x*f(x) - x*sin(tanh(cos(x))) - 3*x

Differentiate $c$ in $x$:

In [9]:
eq = c.diff(x)
eq = simplify(eq, seconds=1)
eq

x*sin(x)*cos(tanh(cos(x)))/cosh(cos(x))**2 + x*Derivative(f(x), x) + f(x) - sin(tanh(cos(x))) - 3

Simplify previous differential equation:

In [10]:
eq = simplify_equa_diff(eq, required=f(x).diff(x))
eq

x*sin(x)*cos(tanh(cos(x))) + x*cosh(cos(x))**2*Derivative(f(x), x) + f(x)*cosh(cos(x))**2 - sin(tanh(cos(x)))*cosh(cos(x))**2 - 3*cosh(cos(x))**2

### Compute Prefix Representations

In [11]:
y_prefix = env.sympy_to_prefix(y)
eq_prefix = env.sympy_to_prefix(eq)
print(f"Solution y with Prefix Notation:\n{y_prefix}\n")
print(f"Differential Equation with Prefix Notation:\n{eq_prefix}")

Solution y with Prefix Notation:
['add', 'INT+', '3', 'add', 'mul', 'c', 'pow', 'x', 'INT-', '1', 'sin', 'tanh', 'cos', 'x']

Differential Equation with Prefix Notation:
['add', 'mul', 'INT-', '3', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'add', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'f', 'x', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'sin', 'tanh', 'cos', 'x', 'add', 'mul', 'x', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'derivative', 'f', 'x', 'x', 'mul', 'x', 'mul', 'cos', 'tanh', 'cos', 'x', 'sin', 'x']


### Encode Input

Clean prefix expressions before they are converted to PyTorch data.

Examples:
- f x  -> Y
- derivative f x x  -> Y'

In [12]:
x1_prefix = env.clean_prefix(eq_prefix)
print(f"Differential Equation Clean Prefix Notation:\n{x1_prefix}")

Differential Equation Clean Prefix Notation:
['add', 'mul', 'INT-', '3', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'add', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'Y', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', 'sin', 'tanh', 'cos', 'x', 'add', 'mul', 'x', 'mul', 'pow', 'cosh', 'cos', 'x', 'INT+', '2', "Y'", 'mul', 'x', 'mul', 'cos', 'tanh', 'cos', 'x', 'sin', 'x']


Create a PyTorch LongTensor for storing $eq$ as a sequence of indexes based on prefix clean notation "words" (Word to
index dictionary is defined inside the Model environment):

In [13]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
x1.transpose(0, 1)

tensor([[ 0, 33, 54, 72, 84, 55, 41, 40, 12, 71, 83, 33, 54, 55, 41, 40, 12, 71,
         83, 78, 33, 54, 72, 82, 54, 55, 41, 40, 12, 71, 83, 64, 69, 40, 12, 33,
         54, 12, 54, 55, 41, 40, 12, 71, 83, 79, 54, 12, 54, 40, 69, 40, 12, 64,
         12,  0]])

Move PyTorch tensors to CUDA (GPU):

In [14]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

Encodes the “meaning” of the input sequence into a single vector, with the Encoder of the Model:

In [15]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[ 0.0103, -0.0965,  0.0125,  ..., -0.0325, -0.0034, -0.0597],
         [ 0.0475, -0.0445, -0.0600,  ..., -0.0021, -0.0066,  0.0205],
         [ 0.0167, -0.0580,  0.0988,  ...,  0.0167,  0.0327, -0.0324],
         ...,
         [-0.2286, -0.2426, -0.0776,  ..., -0.0407,  0.0956,  0.1008],
         [ 0.0002, -0.0553,  0.0610,  ...,  0.0203,  0.0128,  0.0547],
         [ 0.0154, -0.0342,  0.0121,  ..., -0.0616,  0.0113, -0.0232]]],
       device='cuda:0')

### Decode with Beam Search

Instead of picking a single output, a sequence (in this case an hypothesis of differential equation solution), multiple
highly probable choices are retained.

Declare beam size:

In [16]:
beam_size = 10

Takes the encoder output vector and outputs multiple sequences of "words", that in this case should represent the
solution $y$ for the differential equation $eq$, using the Decoder of the model.

In [17]:
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input differential equation $eq$:

In [18]:
eq

x*sin(x)*cos(tanh(cos(x))) + x*cosh(cos(x))**2*Derivative(f(x), x) + f(x)*cosh(cos(x))**2 - sin(tanh(cos(x)))*cosh(cos(x))**2 - 3*cosh(cos(x))**2

Solution $y$ to find:

In [19]:
y

c/x + sin(tanh(cos(x))) + 3

Extract scores and solution hypotheses:

In [20]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Solution Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check if the hypothesis is a valid solution, replacing 'hyp_sympy' with 'f(x)' in the equation
        validation = "YES" if simplify(eq.subs(f(x), hyp_sympy).doit(), seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(hyp_sympy)  + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [21]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

,Score,Solution Hypothesis,Valid
1,-0.040726,$\frac{a_{8}}{x} + \sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + 3$,YES
2,-0.130805,$\sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + \frac{a_{8} + 3 x}{x}$,YES
3,-0.138553,$\frac{a_{8} + x \sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + 3 x}{x}$,YES
4,-0.169402,$\sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + 2 + \frac{a_{8} + x}{x}$,YES
5,-0.174527,$\sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + \frac{3 \left(a_{8} + x\right)}{x}$,YES
6,-0.240685,$3 + \frac{a_{8} + x \sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)}}{x}$,YES
7,-0.241817,$\frac{a_{8} + x \left(\sin{\left(\tanh{\left(\cos{\left(x \right)} \right)} \right)} + 3\right)}{x}$,YES
8,-0.258469,$\frac{a_{8}}{x} + \sin{\left(\sinh{\left(\cos{\left(x \right)} \right)} \right)} + 3$,NO
9,-0.317733,$\frac{a_{8} + x \sin{\left(\sinh{\left(\cos{\left(x \right)} \right)} \right)} + 3 x}{x}$,NO
10,-0.513972,$\frac{a_{8} + x \sin{\left(\sin{\left(\cos{\left(x \right)} \right)} \right)} + 3 x}{x}$,NO


## Second Order Differential Equations (ODE 2)

Procedure:
1. Start from a trivariate function $F(x,c_1,c_2)$, that will be the equation solution, that can be solved in $c_2$.
2. Solve $F(x,c_1,c_2)$ in $c_2$.
3. Differentiate in $x$.
4. Solve in $c_1$.
5. Differentiate in $x$.
6. Simplify the final form.

### Build Environment - Reload Model

Get Trained Model:

In [22]:
model_path = '../models/differential-equations/ode2.pth'
assert os.path.isfile(model_path)

params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

env = build_env(params)
c1 = sympy.Symbol('c1')
c2 = sympy.Symbol('c2')

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare Differential Equation Input and its Solution

Declare a bivariate function $F(x,c_1, c_2)$, that will be the equation solution, that can be solved in $c_2$:

In [23]:
# y_infix = 'c1*exp(x)+c2*exp(-x)'
# y_infix = '(2*x)+((x*(c2+(c1*x)))+(exp(3)))'
# y_infix = '(x**9)*((c1+(c2*((cos(cosh(asin(tanh(x)))))**(-1))))**(-1))'
y_infix = 'c2*(5+((c1+(x+(x**3)))*(exp((-1)*((tan(x))**2)))))'

Converts **y_infix** to a type that can be used inside SymPy:

In [24]:
y = sympy.sympify(y_infix, locals=env.local_dict)
y

c2*((c1 + x**3 + x)*exp(-tan(x)**2) + 5)

Solve $y$ in $c_2$:

In [25]:
solve_c2 = sympy.solve(f(x) - y, c2, check=False, simplify=False)
c2 = solve_c2[0]
c2

-f(x)*exp(tan(x)**2)/(-c1 - x**3 - x - 5*exp(tan(x)**2))

Differentiate $c_2$ in $x$:

In [26]:
eq = c2.diff(x)
eq = simplify(eq, seconds=1)
eq

-(2*tan(x)**2 + 2)*f(x)*exp(tan(x)**2)*tan(x)/(-c1 - x**3 - x - 5*exp(tan(x)**2)) - (3*x**2 + 5*(2*tan(x)**2 + 2)*exp(tan(x)**2)*tan(x) + 1)*f(x)*exp(tan(x)**2)/(-c1 - x**3 - x - 5*exp(tan(x)**2))**2 - exp(tan(x)**2)*Derivative(f(x), x)/(-c1 - x**3 - x - 5*exp(tan(x)**2))

Solve $eq$ in $c_1$:

In [27]:
solve_c1 = sympy.solve(eq, c1, check=False, simplify=False)
c1 = solve_c1[0]
c1

(-2*x**3*f(x)*tan(x)**3 - 2*x**3*f(x)*tan(x) - x**3*Derivative(f(x), x) + 3*x**2*f(x) - 2*x*f(x)*tan(x)**3 - 2*x*f(x)*tan(x) - x*Derivative(f(x), x) + f(x) - 5*exp(tan(x)**2)*Derivative(f(x), x))/(2*f(x)*tan(x)**3 + 2*f(x)*tan(x) + Derivative(f(x), x))

Differentiate $a_8$ in $x$:

In [28]:
eq = c1.diff(x)
eq = simplify(eq, seconds=1)
eq

(-2*x**3*(tan(x)**2 + 1)*f(x) - 2*x**3*(3*tan(x)**2 + 3)*f(x)*tan(x)**2 - 2*x**3*tan(x)**3*Derivative(f(x), x) - 2*x**3*tan(x)*Derivative(f(x), x) - x**3*Derivative(f(x), (x, 2)) - 6*x**2*f(x)*tan(x)**3 - 6*x**2*f(x)*tan(x) - 2*x*(tan(x)**2 + 1)*f(x) - 2*x*(3*tan(x)**2 + 3)*f(x)*tan(x)**2 + 6*x*f(x) - 2*x*tan(x)**3*Derivative(f(x), x) - 2*x*tan(x)*Derivative(f(x), x) - x*Derivative(f(x), (x, 2)) - 5*(2*tan(x)**2 + 2)*exp(tan(x)**2)*tan(x)*Derivative(f(x), x) - 2*f(x)*tan(x)**3 - 2*f(x)*tan(x) - 5*exp(tan(x)**2)*Derivative(f(x), (x, 2)))/(2*f(x)*tan(x)**3 + 2*f(x)*tan(x) + Derivative(f(x), x)) + (-2*(tan(x)**2 + 1)*f(x) - 2*(3*tan(x)**2 + 3)*f(x)*tan(x)**2 - 2*tan(x)**3*Derivative(f(x), x) - 2*tan(x)*Derivative(f(x), x) - Derivative(f(x), (x, 2)))*(-2*x**3*f(x)*tan(x)**3 - 2*x**3*f(x)*tan(x) - x**3*Derivative(f(x), x) + 3*x**2*f(x) - 2*x*f(x)*tan(x)**3 - 2*x*f(x)*tan(x) - x*Derivative(f(x), x) + f(x) - 5*exp(tan(x)**2)*Derivative(f(x), x))/(2*f(x)*tan(x)**3 + 2*f(x)*tan(x) + Derivative(

Simplify previous differential equation:

In [29]:
eq = simplify_equa_diff(eq, required=f(x).diff(x, 2))
eq

12*x**2*f(x)*tan(x)**6 + 42*x**2*f(x)*tan(x)**4 + 36*x**2*f(x)*tan(x)**2 + 6*x**2*f(x) + 12*x**2*tan(x)**3*Derivative(f(x), x) + 12*x**2*tan(x)*Derivative(f(x), x) + 3*x**2*Derivative(f(x), (x, 2)) - 12*x*f(x)*tan(x)**3 - 12*x*f(x)*tan(x) - 6*x*Derivative(f(x), x) + 4*f(x)*tan(x)**6 + 14*f(x)*tan(x)**4 + 12*f(x)*tan(x)**2 + 2*f(x) + 20*exp(tan(x)**2)*tan(x)**6*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)**4*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)**3*Derivative(f(x), (x, 2)) - 20*exp(tan(x)**2)*tan(x)**2*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)*Derivative(f(x), (x, 2)) - 10*exp(tan(x)**2)*Derivative(f(x), x) + 4*tan(x)**3*Derivative(f(x), x) + 4*tan(x)*Derivative(f(x), x) + Derivative(f(x), (x, 2))

### Compute Prefix Representations

In [30]:
y_prefix = env.sympy_to_prefix(y)
eq_prefix = env.sympy_to_prefix(eq)
print(f"Solution y with Prefix Notation:\n{y_prefix}\n")
print(f"Differential Equation with Prefix Notation:\n{eq_prefix}")

Solution y with Prefix Notation:
['mul', 'c2', 'add', 'INT+', '5', 'mul', 'add', 'c1', 'add', 'x', 'pow', 'x', 'INT+', '3', 'exp', 'mul', 'INT-', '1', 'pow', 'tan', 'x', 'INT+', '2']

Differential Equation with Prefix Notation:
['add', 'mul', 'INT+', '2', 'f', 'x', 'add', 'mul', 'INT-', '1', '0', 'mul', 'derivative', 'f', 'x', 'x', 'exp', 'pow', 'tan', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '6', 'mul', 'x', 'derivative', 'f', 'x', 'x', 'add', 'mul', 'INT+', '3', 'mul', 'pow', 'x', 'INT+', '2', 'derivative', 'derivative', 'f', 'x', 'x', 'x', 'add', 'mul', 'INT+', '4', 'mul', 'pow', 'tan', 'x', 'INT+', '3', 'derivative', 'f', 'x', 'x', 'add', 'mul', 'INT+', '4', 'mul', 'pow', 'tan', 'x', 'INT+', '6', 'f', 'x', 'add', 'mul', 'INT+', '4', 'mul', 'derivative', 'f', 'x', 'x', 'tan', 'x', 'add', 'mul', 'INT+', '6', 'mul', 'pow', 'x', 'INT+', '2', 'f', 'x', 'add', 'mul', 'INT+', '1', '2', 'mul', 'pow', 'tan', 'x', 'INT+', '2', 'f', 'x', 'add', 'mul', 'INT+', '1', '4', 'mul', 'pow', 'tan', 'x'

### Encode Input

In [31]:
x1_prefix = env.clean_prefix(eq_prefix)
print(f"Differential Equation Clean Prefix Notation:\n{x1_prefix}")

Differential Equation Clean Prefix Notation:
['add', 'mul', 'INT+', '2', 'Y', 'add', 'mul', 'INT-', '1', '0', 'mul', "Y'", 'exp', 'pow', 'tan', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '6', 'mul', 'x', "Y'", 'add', 'mul', 'INT+', '3', 'mul', 'pow', 'x', 'INT+', '2', "Y''", 'add', 'mul', 'INT+', '4', 'mul', 'pow', 'tan', 'x', 'INT+', '3', "Y'", 'add', 'mul', 'INT+', '4', 'mul', 'pow', 'tan', 'x', 'INT+', '6', 'Y', 'add', 'mul', 'INT+', '4', 'mul', "Y'", 'tan', 'x', 'add', 'mul', 'INT+', '6', 'mul', 'pow', 'x', 'INT+', '2', 'Y', 'add', 'mul', 'INT+', '1', '2', 'mul', 'pow', 'tan', 'x', 'INT+', '2', 'Y', 'add', 'mul', 'INT+', '1', '4', 'mul', 'pow', 'tan', 'x', 'INT+', '4', 'Y', 'add', 'mul', 'INT-', '2', '0', 'mul', 'pow', 'tan', 'x', 'INT+', '2', 'mul', "Y'", 'exp', 'pow', 'tan', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', '2', 'mul', 'x', 'mul', 'pow', 'tan', 'x', 'INT+', '3', 'Y', 'add', 'mul', 'INT-', '1', '2', 'mul', 'x', 'mul', 'Y', 'tan', 'x', 'add', 'mul', 'INT+', '1', '0', 'mul'

In [32]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)

In [33]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

In [34]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[ 0.0388, -0.0235, -0.0223,  ...,  0.0413,  0.0049, -0.0236],
         [ 0.0129, -0.0387,  0.0985,  ...,  0.0268, -0.0467,  0.0007],
         [ 0.1357,  0.0585,  0.0384,  ...,  0.2114, -0.0529, -0.2556],
         ...,
         [-0.3126,  0.1535,  0.3317,  ..., -0.2210, -0.1265,  0.1046],
         [ 0.1923,  0.2024, -0.0650,  ..., -0.1848, -0.1778, -0.1041],
         [ 0.0417, -0.0210, -0.0255,  ...,  0.0377,  0.0020, -0.0247]]],
       device='cuda:0')

### Decode with Beam Search

In [35]:
beam_size = 10

with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input differential equation $eq$:

In [36]:
eq

12*x**2*f(x)*tan(x)**6 + 42*x**2*f(x)*tan(x)**4 + 36*x**2*f(x)*tan(x)**2 + 6*x**2*f(x) + 12*x**2*tan(x)**3*Derivative(f(x), x) + 12*x**2*tan(x)*Derivative(f(x), x) + 3*x**2*Derivative(f(x), (x, 2)) - 12*x*f(x)*tan(x)**3 - 12*x*f(x)*tan(x) - 6*x*Derivative(f(x), x) + 4*f(x)*tan(x)**6 + 14*f(x)*tan(x)**4 + 12*f(x)*tan(x)**2 + 2*f(x) + 20*exp(tan(x)**2)*tan(x)**6*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)**4*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)**3*Derivative(f(x), (x, 2)) - 20*exp(tan(x)**2)*tan(x)**2*Derivative(f(x), x) + 10*exp(tan(x)**2)*tan(x)*Derivative(f(x), (x, 2)) - 10*exp(tan(x)**2)*Derivative(f(x), x) + 4*tan(x)**3*Derivative(f(x), x) + 4*tan(x)*Derivative(f(x), x) + Derivative(f(x), (x, 2))

Solution $y$ to find:

In [37]:
y

c2*((c1 + x**3 + x)*exp(-tan(x)**2) + 5)

Extract scores and solution hypotheses:

In [38]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Solution Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check if the hypothesis is a valid solution, replacing 'hyp_sympy' with 'f(x)' in the equation
        validation = "YES" if simplify(eq.subs(f(x), hyp_sympy).doit(), seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(hyp_sympy)  + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [39]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

,Score,Solution Hypothesis,Valid
1,-0.021180,$a_{9} \left(\left(a_{8} + x^{3} + x\right) e^{- \tan^{2}{\left(x \right)}} + 5\right)$,YES
2,-0.051553,$a_{8} \left(\left(a_{9} + x^{3} + x\right) e^{- \tan^{2}{\left(x \right)}} + 5\right)$,YES
3,-0.144064,$a_{9} \left(\left(a_{8} - x^{3} - x\right) e^{- \tan^{2}{\left(x \right)}} - 5\right)$,YES
4,-0.167011,$a_{8} \left(\left(a_{9} - x^{3} - x\right) e^{- \tan^{2}{\left(x \right)}} - 5\right)$,YES
5,-0.173669,$a_{9} \left(\left(a_{8} + x \left(x^{2} + 1\right)\right) e^{- \tan^{2}{\left(x \right)}} + 5\right)$,YES
6,-0.182952,$a_{9} \left(x \left(\frac{a_{8}}{x} + x^{2} + 1\right) e^{- \tan^{2}{\left(x \right)}} + 5\right)$,YES
7,-0.184083,$a_{9} \left(\frac{\left(a_{8} + x^{3} + x\right) e^{- \tan^{2}{\left(x \right)}}}{5} + 1\right)$,YES
8,-0.205459,$a_{9} \left(3 \left(a_{8} + x^{3} + x\right) e^{- \tan^{2}{\left(x \right)}} + 15\right)$,YES
9,-0.206048,$a_{8} \left(\left(a_{9} + x \left(x^{2} + 1\right)\right) e^{- \tan^{2}{\left(x \right)}} + 5\right)$,YES
10,-0.588345,$a_{9} \left(- a_{8} + x^{3} + x + 5\right)$,NO
